## 多输入通道

当输入包含多个通道时，需要构造和通道数相同的卷积核，例如彩色图片有RGB三个通道，因此构建三个卷积核，分别对每个通道进行卷积运算，然后将结果合并在一起。

![](img/conv-multi-in.png)

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 分别对每个通道进行计算，然后将结果加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [2]:
X = torch.tensor([[[0, 1, 2], [3, 4, 5], [6, 7, 8]], 
                  [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = torch.tensor([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 多输出通道

目前为止无论多少输入通道，都只有一个输出通道，现实中多个通道堆模型的训练至关重要，可以把每个输出通道理解为对不同特征的响应。

用$c_i$和$c_o$分别表示输入和输出通道的数量，需要为每个输出通道创建$c_i \times k_h \times k_w$的卷积核张量，那么完整的卷积核为$c_o \times c_i \times k_h \times k_w$。

In [3]:
def corr2d_multi_in_out(X, K):
    # 分别计算每个输出通道，然后进行拼接
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [4]:
K = torch.stack((K, K+1, K+2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 1$\times$1卷积层

1$\times$1的卷积层失去了卷积的本质，但是1$\times$卷积仍然很流行，通常包含在复杂的深度网络设计中。

1$\times$1的计算只发生在通道上，可以看作是从输入层$c_i$到输出层$c_o$的全连接层。

![](img/conv-1x1.png)

In [6]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    return (K @ X).reshape((c_o, h, w))

In [7]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

assert float(torch.abs(Y1 - Y2).sum()) < 1e-6